# Ques.5 Predict the sentiment of tweets by followers of naveen_odisha

In [17]:
import json
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from textblob import TextBlob
from elasticsearch import Elasticsearch

# import twitter keys and tokens
from config import *

# create instance of elasticsearch
es = Elasticsearch()

TWITTER_CONSUMER_KEY = 'CONSUMER KEY'
TWITTER_CONSUMER_SECRET = 'CONSUMER SECRET'
TWITTER_ACCESS_TOKEN = 'ACCESS TOKEN'
TWITTER_ACCESS_TOKEN_SECRET = 'ACCESS TOKEN SECRET'


class TweetStreamListener(StreamListener):

    # on success
    def on_data(self, data):

        # decode json
        dict_data = json.loads(data)

        # pass tweet into TextBlob
        tweet = TextBlob(dict_data["text"])

        # output sentiment polarity
        print tweet.sentiment.polarity

        # determine if sentiment is positive, negative, or neutral
        if tweet.sentiment.polarity < 0:
            sentiment = "negative"
        elif tweet.sentiment.polarity == 0:
            sentiment = "neutral"
        else:
            sentiment = "positive"

        # output sentiment
        print sentiment

        # add text and sentiment info to elasticsearch
        es.index(index="sentiment",
                 doc_type="test-type",
                 body={"author": dict_data["user"]["screen_name"],
                       "date": dict_data["created_at"],
                       "message": dict_data["text"],
                       "polarity": tweet.sentiment.polarity,
                       "subjectivity": tweet.sentiment.subjectivity,
                       "sentiment": sentiment})
        return True

    # on failure
    def on_error(self, status):
        print status

if __name__ == '__main__':

    # create instance of the tweepy tweet stream listener
    listener = TweetStreamListener()

    # set twitter keys/tokens
    auth = OAuthHandler(TWITTER_CONSUMER_KEY,TWITTER_CONSUMER_SECRET)
    auth.set_access_token(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET)

    # create instance of the tweepy stream
    stream = Stream(auth, listener)

    # search twitter for "congress" keyword
    stream.filter(track=['naveen_odisha'])

Traceback (most recent call last):
  File "C:\Users\test\Anaconda2\lib\site-packages\elasticsearch\connection\http_urllib3.py", line 91, in perform_request
    response = self.pool.urlopen(method, url, body, retries=False, headers=self.headers, **kw)
  File "C:\Users\test\Anaconda2\lib\site-packages\urllib3\connectionpool.py", line 608, in urlopen
    _stacktrace=sys.exc_info()[2])
  File "C:\Users\test\Anaconda2\lib\site-packages\urllib3\util\retry.py", line 224, in increment
    raise six.reraise(type(error), error, _stacktrace)
  File "C:\Users\test\Anaconda2\lib\site-packages\urllib3\connectionpool.py", line 558, in urlopen
    body=body, headers=headers)
  File "C:\Users\test\Anaconda2\lib\site-packages\urllib3\connectionpool.py", line 353, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "C:\Users\test\Anaconda2\lib\httplib.py", line 1057, in request
    self._send_request(method, url, body, headers)
  File "C:\Users\test\Anaconda2\lib\httplib.py", line

0.0
neutral


ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x000000001D517BA8>: Failed to establish a new connection: [Errno 10061] No connection could be made because the target machine actively refused it) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x000000001D517BA8>: Failed to establish a new connection: [Errno 10061] No connection could be made because the target machine actively refused it)